# Try downloading UKB data

+ Link: https://pan-dev.ukbb.broadinstitute.org/docs/hail-format/index.html#extracting-a-subset-of-ld-matrix
+ European samples are the largest, totaling 14.1T of data

In [1]:
using Revise
using gnomAD

First check how many files there are:

In [10]:
get_ukb_filenames("EUR", join=false)

124281-element Vector{String}:
 "part-000000-44-0-0-22f828c8-17c3-7c3c-1fa1-1fc113144aca"
 "part-000001-44-1-0-d35bc353-c9b7-28e7-315b-f1bcd8d9e50b"
 "part-000002-44-2-2-7dc4e8fd-75f7-26d0-4b9c-65628f25cf34"
 "part-000003-44-3-0-74be7ee3-ba85-f9ce-4add-9638cf387f8a"
 "part-000004-44-4-0-b11cbd6a-50e2-4e5e-ae70-932ff3aebd5f"
 "part-000005-44-5-0-104f3537-b7e4-0959-555b-7e65ca2b00ae"
 "part-000006-44-6-0-f99bebaa-7361-9c3e-65a0-2700858c2b28"
 "part-000007-44-7-0-90293a19-7ae6-f6a6-1141-48f62ee6f8b1"
 "part-000008-44-8-0-847d7052-f784-0c5f-c4f8-d16df8745a23"
 "part-000009-44-9-0-2b61d351-aeda-4c03-52aa-cb380599219d"
 "part-000010-44-10-0-90b9d808-fa98-2727-ca3e-d63676464185"
 "part-000011-44-11-0-7a15dbc9-b5c7-3500-8258-886bbf569e35"
 "part-000012-44-12-0-73b49602-7c17-3f3b-860c-6aaa8b25f36b"
 ⋮
 "part-124226-44-124226-0-9948291d-f1c5-775a-4ed3-7d75c9e7d007"
 "part-124227-44-124227-0-43ffd695-b0c7-577e-0d5a-8ea5f3ed84d0"
 "part-124228-44-124228-0-d22a4e30-04f2-82aa-dfd2-4d3ae95e57b4"
 "pa

Try downloading the first 10 files (~1GB)

In [12]:
population = "EUR"
outdir = "/Users/biona001/.julia/dev/gnomAD/data"
download_ukb_LD_matrices(population, outdir, start_from=1, num_files=10)

Progress: 100%|█████████████████████████████████████████| Time: 0:03:48


Downloading 10 files took 228 seconds, so download all 124k files would take roughly 30 days. Fortunately, the `start_from` and `num_files` keywords allow us to split this into 30 jobs, so we should be able to download everything in 1 day or so.

# Reading the LD panel

In [2]:
data = "/Users/biona001/.julia/dev/gnomAD/data/UKBB.EUR.ldadj.bm"
bm = hail_block_matrix(data);

Check size of matrix

In [3]:
size(bm)

2023-03-02 18:35:57.904 WARN  NativeCodeLoader:60 - Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


Initializing Hail with default parameters...
Picked up _JAVA_OPTIONS: -Xmx16g
Picked up _JAVA_OPTIONS: -Xmx16g
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
Running on Apache Spark version 3.1.3
SparkUI available at http://dn52ecmi.sunet:4040
Welcome to
     __  __     <>__
    / /_/ /__  __/ /
   / __  / _ `/ / /
  /_/ /_/\_,_/_/_/   version 0.2.109-b71b065e4bb6
LOGGING: writing to /Users/biona001/.julia/dev/gnomAD/test/hail-20230302-1835-0.2.109-b71b065e4bb6.log


(23960350, 23960350)

Read first 10000 by 10000 block into memory

In [4]:
Sigma = bm[1:10000, 1:10000]

10000×10000 Matrix{Float64}:
 0.99996  0.00098559  -0.000830543  …  -0.000520665   0.00422904
 0.0      0.999942    -0.00104453      -0.00494589    0.00050959
 0.0      0.0          0.999963        -0.00129536    0.00216722
 0.0      0.0          0.0             -0.00663755    0.000142521
 0.0      0.0          0.0              0.0104731     0.000936687
 0.0      0.0          0.0          …  -0.00202249   -0.000497824
 0.0      0.0          0.0             -0.00550819    0.00283421
 0.0      0.0          0.0             -0.00145102   -0.000761935
 0.0      0.0          0.0              0.00287458   -0.0010154
 0.0      0.0          0.0             -0.000450895  -0.00130722
 0.0      0.0          0.0          …  -0.00101125    0.000451512
 0.0      0.0          0.0              0.00590455   -0.00110266
 0.0      0.0          0.0             -0.00274252   -0.000817142
 ⋮                                  ⋱                
 0.0      0.0          0.0             -0.109458     -0.00866031
 0

Check if the given block is PSD by computing its eigenvalues

In [5]:
using LinearAlgebra
eigvals(Symmetric(Sigma)) # Symmetric uses upper triangular portion of data

10000-element Vector{Float64}:
  -4.496303144059796e-15
  -1.762987426134658e-15
  -1.040030990442209e-15
  -4.2089306816541705e-16
  -3.960324780237165e-16
  -3.9060196040323796e-16
  -2.985734642577166e-16
  -2.8601497287222336e-16
  -2.5553650845256305e-16
  -1.998130772222011e-16
  -1.8449274904356797e-16
  -1.7028015609869147e-16
  -1.3192630645503314e-16
   ⋮
 116.57001725005479
 124.37732317876345
 125.11477060743002
 135.80425399688727
 142.93300594996916
 149.24000792252738
 161.47927858350278
 181.82233513932786
 228.3300216753156
 235.961635959381
 280.6540947869515
 322.07381500507836

## Misc

Some miscellaneous checks

In [3]:
using AWS: @service
using AWSS3

In [4]:
p = S3Path("s3://pan-ukb-us-east-1/ld_release/")

p"s3://pan-ukb-us-east-1/ld_release/"

In [5]:
readdir(p)

139-element Vector{String}:
 "HM3.UKBB.AFR.qc.snplist.ht/"
 "HM3.UKBB.AMR.qc.snplist.ht/"
 "HM3.UKBB.CSA.qc.snplist.ht/"
 "HM3.UKBB.EAS.qc.snplist.ht/"
 "HM3.UKBB.EUR.qc.snplist.ht/"
 "HM3.UKBB.MID.qc.snplist.ht/"
 "UKBB.AFR.25LDMS.l2.M"
 "UKBB.AFR.25LDMS.l2.M_5_50"
 "UKBB.AFR.25LDMS.l2.annot.gz"
 "UKBB.AFR.25LDMS.l2.ldscore.gz"
 "UKBB.AFR.25LDMS.ldscore.ht/"
 "UKBB.AFR.25LDMS.rsid.l2.annot.gz"
 "UKBB.AFR.25LDMS.rsid.l2.ldscore.gz"
 ⋮
 "UKBB.MID.8LDMS.l2.ldscore.gz"
 "UKBB.MID.8LDMS.ldscore.ht/"
 "UKBB.MID.8LDMS.rsid.l2.annot.gz"
 "UKBB.MID.8LDMS.rsid.l2.ldscore.gz"
 "UKBB.MID.l2.M"
 "UKBB.MID.l2.M_5_50"
 "UKBB.MID.l2.ldscore.gz"
 "UKBB.MID.ldadj.bm/"
 "UKBB.MID.ldadj.variant.b38.ht/"
 "UKBB.MID.ldadj.variant.ht/"
 "UKBB.MID.ldscore.ht/"
 "UKBB.MID.rsid.l2.ldscore.gz"

In [7]:
file = joinpath(p, "UKBB.EUR.ldadj.bm")
stat(file)

Status(
  device = 0,
  inode = 0,
  mode = -rw-rw-rw-,
  nlink = 0,
  uid = 501 (biona001),
  gid = 20 (staff),
  rdev = 0,
  size = 0 (0.0),
  blksize = 4096 (4.0K),
  blocks = 0,
  mtime = 0000-01-01T00:00:00,
  ctime = 0000-01-01T00:00:00,
)

## List all files available

There are 124283 objects, totaling 14.1T of data.

In [ ]:
run(`aws s3 ls --summarize --human-readable --recursive s3://pan-ukb-us-east-1/ld_release/UKBB.EUR.ldadj.bm`)